<a href="https://colab.research.google.com/github/cbologa/public_databases/blob/main/R/ChEMBL_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Latest ChEMBL database data dictionary is available at https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/schema_documentation.html

Load required packages

In [ ]:
if(!require(pacman)) install.packages("pacman")
pacman::p_load(R.utils, RSQLite, curl)
library(DBI)

Download and unpack the SQLite version of ChEMBL database

In [2]:
zip  <- "chembl_31_sqlite.tar.gz"
file <- "chembl_31/chembl_31_sqlite/chembl_31.db"
if(!file.exists(file)){
   curl_download(paste0("ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/",zip),zip)
   untar(zip,files = file, exdir = "./")
}

Connect to database

In [3]:
con <- dbConnect(RSQLite::SQLite(), file)

Show all tables in the database

In [ ]:
dbListTables(con)

Show fields in a table/view

In [ ]:
dbListFields(con, "activities")

Extract references, bioactivities, and compounds tested on equine encephalitis viruses

In [ ]:
df1 <- dbGetQuery(con, "
select
  target_dictionary.*,
  assays.*,
  activities.*,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.*,
  docs.*
from
  target_dictionary
  left join assays on target_dictionary.tid=assays.tid
  left join docs on assays.doc_id=docs.doc_id
  left join activities on assays.assay_id=activities.assay_id
  left join molecule_dictionary on activities.molregno=molecule_dictionary.molregno
  left join compound_structures on molecule_dictionary.molregno=compound_structures.molregno
where 
  target_dictionary.organism like '%equine encephalitis virus'
group by
  target_dictionary.tid,
  assays.assay_id,
  activities.activity_id,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.molregno,
  docs.doc_id
order by
  docs.year,
  docs.doi
")

#save results to file
write.csv(df1, "chembl_eev.csv", row.names=FALSE)

# Show results
df1

Close database connection

In [7]:
dbDisconnect(con)